Import

In [74]:
import json, string, re
from collections import defaultdict
from datetime import datetime as dt
import numpy as np
import pandas as pd
import seaborn as sns

from collections import defaultdict 
import nltk
import unicodedata

nltk.download('wordnet')
nltk.download('stopwords')

data = json.load(open('data/realdonaldtrump.json', 'r'))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Prepare Variables

In [75]:
tweets = []
for key, value in data.items():
    value['id'] = key
    value['created_at'] = dt.strptime(value['created_at'], '%a %b %d %H:%M:%S %z %Y')
    tweets.append(value)

df = pd.DataFrame(tweets)
df.head()

,created_at,full_text,retweet_count,favorite_count,id
0,2018-08-01 21:04:18+00:00,Join me tomorrow night at 7pmE in Wilkes-Barre...,8961,38653,1024762772733796353
1,2020-03-24 02:44:49+00:00,....us since the beginning of this crisis.” Th...,13897,68761,1242281185084149762
2,2015-06-16 15:57:23+00:00,I am officially running for President of the U...,10006,10640,610838591242137600
3,2018-02-20 13:38:49+00:00,"I have been much tougher on Russia than Obama,...",12569,65832,965943827931549696
4,2020-09-01 21:59:03+00:00,Not even a “minor” one. Thank you! https://t.c...,9529,37794,1300916103229526017


old functions

In [76]:
def plot_count_by_time():
    data = df['created_at'].apply(lambda ts: (ts.hour+24-5)%24)
    sns.histplot(data=data)

current playground

In [84]:
ADDITIONAL_STOPWORDS = ['amp']

def clean(text):
  """
  A simple function to clean up the data. All the words that
  are not designated as a stop word is then lemmatized after
  encoding and basic regex parsing are performed.
  """
  wnl = nltk.stem.WordNetLemmatizer()
  stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
  text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
  words = re.sub(r'[^\w\s]', '', text).split()
  return [wnl.lemmatize(word) for word in words if word not in stopwords]

texts = df['full_text'].apply(lambda text: text.lower())
# lowercase
# remove
word_count = defaultdict(lambda: 0)
for text in texts:
    # # get rid of links and usernames
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', text)
    text = re.sub("(@[A-Za-z0-9_]+)","", text)
    # match = re.match("[A-Za-z_]+", text)
    # print(match)
    for word in clean(text):
        word_count[word] += 1

data = pd.DataFrame(columns=['word', 'count'])
for k, v in word_count.items():
    data = data.append({'word': k, 'count': v}, ignore_index=True)
data.sort_values(by=['count'], ascending=False).head()
# print(word_count)

,word,count
9,great,683
53,thank,582
242,trump,327
98,amp,300
8,america,288
